In [ ]:
import netCDF4 as nc
import numpy as np
from scipy.interpolate import interp1d
import xarray as xr
import xroms

z_r = np.load('/srv/scratch/z5297792/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))

fnumbers = [f'{num:05}' for num in range(1461, 10611+1, 30)] # last valid file 10611

H = 1000
for f, fnumber in enumerate(fnumbers):
    fname = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_'+str(fnumber)+'.nc' 
    ds = xr.open_dataset(fname)
    N2 = ds.xroms.N2
    
    N2 = np.transpose(N2.data, axes=(3, 2, 1, 0))
    N2 = np.flip(N2, axis=2)
    
    i, j, k = np.where((-z_r) < H)
    
    N2[i, j, k, :] = np.nan
    
    N2_bar = np.nanmean(N2, axis=2) 
    
    np.save('/srv/scratch/z5297792/Climatology/N2s/N2bar_'+str(fnumber)+'.npy', N2_bar)

    if f % 10 == 0:
        print(fnumber)


/scratch/pbs.6905923.kman.restech.unsw.edu.au/ipykernel_3876157/3916481685.py:25: RuntimeWarning: Mean of empty slice
  N2_bar = np.nanmean(N2, axis=2)


01461


/scratch/pbs.6905923.kman.restech.unsw.edu.au/ipykernel_3876157/3916481685.py:25: RuntimeWarning: Mean of empty slice
  N2_bar = np.nanmean(N2, axis=2)
/scratch/pbs.6905923.kman.restech.unsw.edu.au/ipykernel_3876157/3916481685.py:25: RuntimeWarning: Mean of empty slice
  N2_bar = np.nanmean(N2, axis=2)
/scratch/pbs.6905923.kman.restech.unsw.edu.au/ipykernel_3876157/3916481685.py:25: RuntimeWarning: Mean of empty slice
  N2_bar = np.nanmean(N2, axis=2)
